In [1]:
!pip install deepvision-toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import deepvision
import torch
import tensorflow as tf

config = {
    'batch_size': 16,
    'epochs': 5
}

# Pre-Train on Imagenette using DeepVision and TensorFlow

To simulate larger-scale pre-training on a dataset such as Imagenet, or a proprietary dataset.

In [3]:
import tensorflow_datasets as tfds

(train_set, test_set), info = tfds.load("imagenette", 
                                           split=["train", "validation"],
                                           as_supervised=True, with_info=True)

class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

def preprocess_img(img, label):
    img = tf.image.resize(img, (224, 224))
    img = img/255
    return img, label

train_set = train_set.map(preprocess_img).batch(config['batch_size']).prefetch(tf.data.AUTOTUNE)
test_set = test_set.map(preprocess_img).batch(config['batch_size'], drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [4]:
tf_model = deepvision.models.EfficientNetV2B0(include_top=True,
                                       classes=10,
                                       input_shape=(224, 224, 3),
                                       backend='tensorflow')

tf_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
  metrics=['accuracy']
)
history = tf_model.fit(train_set, epochs=5, validation_data=test_set)

Epoch 1/5
592/592 [==============================] - 154s 160ms/step - loss: 2.1109 - accuracy: 0.2252 - val_loss: 1.9850 - val_accuracy: 0.2913
Epoch 2/5
592/592 [==============================] - 83s 140ms/step - loss: 1.8669 - accuracy: 0.3478 - val_loss: 1.8344 - val_accuracy: 0.3612
Epoch 3/5
592/592 [==============================] - 92s 156ms/step - loss: 1.7025 - accuracy: 0.4103 - val_loss: 1.7361 - val_accuracy: 0.4099
Epoch 4/5
592/592 [==============================] - 83s 140ms/step - loss: 1.5341 - accuracy: 0.4758 - val_loss: 1.6392 - val_accuracy: 0.4551
Epoch 5/5
592/592 [==============================] - 82s 139ms/step - loss: 1.3637 - accuracy: 0.5385 - val_loss: 1.5948 - val_accuracy: 0.4778


In [5]:
tf_model.save('effnet.h5')

# Transfer TensorFlow Weights to PyTorch

Using DeepVision models, you can easily pick up a trained checkpoint of a model trained in *either* TensorFlow or PyTorch, and fine-tune with either libraries/ecosystems. Each model comes with its `model_weight_mapper` which exposes a `load()` function, accepting a `filepath` to the trained model, the `origin` library and `target` library.

You'll need to supply a dummy input in the *origin library's* expected format, and DeepVision will reconstruct the target library's equivalent of the model, with the loaded and converted weights, ready to fine-tune:

In [6]:
from deepvision.models.classification.efficientnet import efficientnet_weight_mapper

In [7]:
dummy_input = tf.random.normal([1, 224, 224, 3])
pt_model = efficientnet_weight_mapper.load_tf_to_pt(filepath='effnet.h5',
                                           dummy_input=dummy_input)

# If the top classes differ, you can set them here
pt_model.top_dense = torch.nn.Linear(pt_model.top_dense.in_features, 10)

# Fine-Tune in PyTorch

In [8]:
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import pytorch_lightning as pl

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Resize([224, 224])])

cifar_train = CIFAR10('cifar10', train=True, download=True, transform=transform)
cifar_test = CIFAR10('cifar10', train=False, download=True, transform=transform)

train_dataloader = DataLoader(cifar_train, batch_size=config['batch_size'], drop_last=True)
val_dataloader = DataLoader(cifar_test, batch_size=config['batch_size'], drop_last=True)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pt_model.parameters(), 1e-5)

pt_model.compile(loss=loss, optimizer=optimizer)
trainer = pl.Trainer(accelerator=device, max_epochs=5)
trainer.fit(pt_model, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | acc       | MulticlassAccuracy | 0     
1 | stem_conv | Conv2d             | 864   
2 | stem_bn   | BatchNorm2d        | 64    
3 | blocks    | ModuleList         | 5.6 M 
4 | top_conv  | Conv2d             | 245 K 
5 | top_bn    | BatchNorm2d        | 2.6 K 
6 | top_dense | Linear             | 12.8 K
7 | loss      | CrossEntropyLoss   | 0     
-------------------------------------------------
5.8 M     Trainable params
60.6 K    Non-trainabl

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


# Pre-Train using PyTorch

In [10]:
import torch

from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import RichProgressBar

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Resize([224, 224])])

cifar_train = CIFAR10('cifar10', train=True, download=True, transform=transform)
cifar_test = CIFAR10('cifar10', train=False, download=True, transform=transform)

train_dataloader = DataLoader(cifar_train, batch_size=config['batch_size'], drop_last=True)
val_dataloader = DataLoader(cifar_test, batch_size=config['batch_size'], drop_last=True)

pt_model = deepvision.models.EfficientNetV2B0(include_top=True,
                                       classes=10,
                                       input_shape=(3, 224, 224),
                                       backend='pytorch')

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pt_model.parameters(), 1e-5)

pt_model.compile(loss=loss, optimizer=optimizer)
trainer = pl.Trainer(accelerator=device, max_epochs=config['epochs'])
trainer.fit(pt_model, train_dataloader, val_dataloader)

Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | acc       | MulticlassAccuracy | 0     
1 | stem_conv | Conv2d             | 864   
2 | stem_bn   | BatchNorm2d        | 64    
3 | blocks    | ModuleList         | 5.6 M 
4 | top_conv  | Conv2d             | 245 K 
5 | top_bn    | BatchNorm2d        | 2.6 K 
6 | top_dense | Linear             | 12.8 K
7 | loss      | CrossEntropyLoss   | 0     
-------------------------------------------------
5.9 M     Trainable params
0         Non-trainabl

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [11]:
torch.save(pt_model.state_dict(), 'effnet.pt')

# Transfer PyTorch Weights to TensorFlow

In [12]:
from deepvision.models.classification.efficientnet import efficientnet_weight_mapper

dummy_input_torch = torch.ones(1, 3, 224, 224)
kwargs = {'include_top': True, 'classes': 10, 'input_shape':(3, 224, 224)}
tf_model = efficientnet_weight_mapper.load_pt_to_tf(filepath='effnet.pt',
                                architecture='EfficientNetV2B0',
                                kwargs=kwargs,
                                dummy_input=dummy_input_torch)

# Fine-Tune in TensorFlow

In [13]:
import tensorflow_datasets as tfds
import tensorflow as tf

(train_set, test_set), info = tfds.load("imagenette", 
                                           split=["train", "validation"],
                                           as_supervised=True, with_info=True)

class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

def preprocess_img(img, label):
    img = tf.image.resize(img, (224, 224))
    img = img/255
    return img, label

train_set = train_set.map(preprocess_img).batch(config['batch_size']).prefetch(tf.data.AUTOTUNE)
test_set = test_set.map(preprocess_img).batch(config['batch_size'], drop_remainder=True).prefetch(tf.data.AUTOTUNE)

tf_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
  metrics=['accuracy']
)
history = tf_model.fit(train_set, epochs=5, validation_data=test_set)

Epoch 1/5
592/592 [==============================] - 133s 157ms/step - loss: 2.3837 - accuracy: 0.2866 - val_loss: 1.7683 - val_accuracy: 0.4000
Epoch 2/5
592/592 [==============================] - 82s 138ms/step - loss: 1.6409 - accuracy: 0.4462 - val_loss: 1.5722 - val_accuracy: 0.4658
Epoch 3/5
592/592 [==============================] - 82s 137ms/step - loss: 1.4346 - accuracy: 0.5211 - val_loss: 1.4620 - val_accuracy: 0.5148
Epoch 4/5
592/592 [==============================] - 81s 137ms/step - loss: 1.2633 - accuracy: 0.5808 - val_loss: 1.4051 - val_accuracy: 0.5436
Epoch 5/5
592/592 [==============================] - 91s 153ms/step - loss: 1.0937 - accuracy: 0.6404 - val_loss: 1.4067 - val_accuracy: 0.5508
